In [24]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV


import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [3]:
%matplotlib inline

In [4]:
train = pd.read_csv('/Users/ryan/Downloads/playground-series-s6e1/train.csv')
test = pd.read_csv('/Users/ryan/Downloads/playground-series-s6e1/test.csv')
submit = pd.read_csv('/Users/ryan/Downloads/playground-series-s6e1/sample_submission.csv')

In [5]:
train.head(5)

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                630000 non-null  int64  
 1   age               630000 non-null  int64  
 2   gender            630000 non-null  object 
 3   course            630000 non-null  object 
 4   study_hours       630000 non-null  float64
 5   class_attendance  630000 non-null  float64
 6   internet_access   630000 non-null  object 
 7   sleep_hours       630000 non-null  float64
 8   sleep_quality     630000 non-null  object 
 9   study_method      630000 non-null  object 
 10  facility_rating   630000 non-null  object 
 11  exam_difficulty   630000 non-null  object 
 12  exam_score        630000 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 62.5+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270000 entries, 0 to 269999
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                270000 non-null  int64  
 1   age               270000 non-null  int64  
 2   gender            270000 non-null  object 
 3   course            270000 non-null  object 
 4   study_hours       270000 non-null  float64
 5   class_attendance  270000 non-null  float64
 6   internet_access   270000 non-null  object 
 7   sleep_hours       270000 non-null  float64
 8   sleep_quality     270000 non-null  object 
 9   study_method      270000 non-null  object 
 10  facility_rating   270000 non-null  object 
 11  exam_difficulty   270000 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 24.7+ MB


In [8]:
train.describe()

,id,age,study_hours,class_attendance,sleep_hours,exam_score
count,630000.000000,630000.000000,630000.000000,630000.000000,630000.000000,630000.000000
mean,314999.500000,20.545821,4.002337,71.987261,7.072758,62.506672
std,181865.479132,2.260238,2.359880,17.430098,1.744811,18.916884
min,0.000000,17.000000,0.080000,40.600000,4.100000,19.599000
25%,157499.750000,19.000000,1.970000,57.000000,5.600000,48.800000
50%,314999.500000,21.000000,4.000000,72.600000,7.100000,62.600000
75%,472499.250000,23.000000,6.050000,87.200000,8.600000,76.300000
max,629999.000000,24.000000,7.910000,99.400000,9.900000,100.000000


In [9]:
test.describe()

,id,age,study_hours,class_attendance,sleep_hours
count,270000.000000,270000.000000,270000.000000,270000.000000,270000.000000
mean,764999.500000,20.544137,4.003878,71.982509,7.072070
std,77942.430678,2.260452,2.357741,17.414695,1.745513
min,630000.000000,17.000000,0.080000,40.600000,4.100000
25%,697499.750000,19.000000,1.980000,57.000000,5.600000
50%,764999.500000,21.000000,4.010000,72.600000,7.100000
75%,832499.250000,23.000000,6.050000,87.200000,8.600000
max,899999.000000,24.000000,7.910000,99.400000,9.900000


In [10]:
y_train = train['exam_score']
train_x = train.iloc[:,:-1]

In [11]:
data = pd.concat([train_x, test])
data

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy
...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,21,other,b.com,2.55,82.3,yes,8.4,average,mixed,medium,hard
269996,899996,17,female,b.com,0.49,46.4,yes,8.8,good,mixed,low,easy
269997,899997,22,male,bba,6.62,74.7,yes,5.5,good,coaching,high,easy
269998,899998,22,other,ba,4.08,51.8,yes,8.7,poor,online videos,high,moderate


In [12]:
data.reset_index(inplace=True, drop=True)
data

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy
...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899995,21,other,b.com,2.55,82.3,yes,8.4,average,mixed,medium,hard
899996,899996,17,female,b.com,0.49,46.4,yes,8.8,good,mixed,low,easy
899997,899997,22,male,bba,6.62,74.7,yes,5.5,good,coaching,high,easy
899998,899998,22,other,ba,4.08,51.8,yes,8.7,poor,online videos,high,moderate


In [13]:
data['internet_access'] = data['internet_access'].astype('category').cat.codes

data

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,0,21,female,b.sc,7.91,98.8,0,4.9,average,online videos,low,easy
1,1,18,other,diploma,4.95,94.8,1,4.7,poor,self-study,medium,moderate
2,2,20,female,b.sc,4.68,92.6,1,5.8,poor,coaching,high,moderate
3,3,19,male,b.sc,2.00,49.5,1,8.3,average,group study,high,moderate
4,4,23,male,bca,7.65,86.9,1,9.6,good,self-study,high,easy
...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899995,21,other,b.com,2.55,82.3,1,8.4,average,mixed,medium,hard
899996,899996,17,female,b.com,0.49,46.4,1,8.8,good,mixed,low,easy
899997,899997,22,male,bba,6.62,74.7,1,5.5,good,coaching,high,easy
899998,899998,22,other,ba,4.08,51.8,1,8.7,poor,online videos,high,moderate


In [14]:
data['sleep_quality'].unique()

array(['average', 'poor', 'good'], dtype=object)

In [15]:
mapping_sleep_quality = {
    'poor': 0,
    'average': 1,
    'good': 2
}

mapping_facility_rating = {
    'low': 0,
    'medium': 1,
    'high': 2
}

mapping_exam_difficulty = {
    'easy': 0,
    'moderate': 1,
    'hard': 2
}
data['sleep_quality'] = data['sleep_quality'].map(mapping_sleep_quality)
data['facility_rating'] = data['facility_rating'].map(mapping_facility_rating)
data['exam_difficulty'] = data['exam_difficulty'].map(mapping_exam_difficulty)
data

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,0,21,female,b.sc,7.91,98.8,0,4.9,1,online videos,0,0
1,1,18,other,diploma,4.95,94.8,1,4.7,0,self-study,1,1
2,2,20,female,b.sc,4.68,92.6,1,5.8,0,coaching,2,1
3,3,19,male,b.sc,2.00,49.5,1,8.3,1,group study,2,1
4,4,23,male,bca,7.65,86.9,1,9.6,2,self-study,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899995,21,other,b.com,2.55,82.3,1,8.4,1,mixed,1,2
899996,899996,17,female,b.com,0.49,46.4,1,8.8,2,mixed,0,0
899997,899997,22,male,bba,6.62,74.7,1,5.5,2,coaching,2,0
899998,899998,22,other,ba,4.08,51.8,1,8.7,0,online videos,2,1


In [16]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded = ohe.fit_transform(data[['gender', 'course', 'study_method']])

encoded_df = pd.DataFrame(
    encoded,
    columns=ohe.get_feature_names_out(['gender', 'course', 'study_method'])
)

data = data.drop(columns=['gender', 'course', 'study_method']).reset_index(drop=True)
data = pd.concat([data, encoded_df], axis=1)

data


,id,age,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,facility_rating,exam_difficulty,gender_female,...,course_b.tech,course_ba,course_bba,course_bca,course_diploma,study_method_coaching,study_method_group study,study_method_mixed,study_method_online videos,study_method_self-study
0,0,21,7.91,98.8,0,4.9,1,0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,18,4.95,94.8,1,4.7,0,1,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,20,4.68,92.6,1,5.8,0,2,1,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,19,2.00,49.5,1,8.3,1,2,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,23,7.65,86.9,1,9.6,2,2,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899995,21,2.55,82.3,1,8.4,1,1,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
899996,899996,17,0.49,46.4,1,8.8,2,0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
899997,899997,22,6.62,74.7,1,5.5,2,2,0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
899998,899998,22,4.08,51.8,1,8.7,0,2,1,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
X_train = data.iloc[:len(train_x), :]
X_test = data.iloc[len(train_x):, :]

In [25]:
xgb_model = xgb.XGBRegressor(
    random_state=42,
    n_jobs=-1
)

param_grid = {
    'n_estimators': [500 ,700 ,900],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Best params:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)



Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=1.0; total time=  11.9s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=1.0; total time=  12.1s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=1.0; total time=  12.1s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.8; total time=  12.3s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.8; total time=  12.6s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.6; total time=  12.8s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.8; total time=  12.7s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=3, n_estimators=500, subsample=0.6; total time=  13.1s
[CV] END 

/opt/anaconda3/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=500, subsample=0.6; total time=  11.9s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=900, subsample=0.6; total time=  46.2s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=500, subsample=0.6; total time=  12.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=900, subsample=0.8; total time=  45.4s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=500, subsample=0.8; total time=  11.3s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=900, subsample=0.8; total time=  44.5s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=500, subsample=0.8; total time=  11.4s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=500, subsample=1.0; total time=  10.4s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estima

In [26]:
best_model = grid_search.best_estimator_

In [27]:
predsTest = best_model.predict(X_test)

In [28]:
test['exam_score'] = predsTest

submission = test[['id', 'exam_score']]
submission.to_csv('score_prediction_XGBoost.csv', index=False)
submission

,id,exam_score
0,630000,72.430832
1,630001,69.451141
2,630002,87.700661
3,630003,54.799206
4,630004,47.224792
...,...,...
269995,899995,59.802982
269996,899996,39.777893
269997,899997,90.217331
269998,899998,54.792103
